# Setup

In [31]:
# Import everything and setup things
import sys
import os
file_folder = os.getcwd()
if "testing_folder" in file_folder:
    file_folder = os.path.dirname(file_folder)
    sys.path.append(file_folder)
    os.chdir(file_folder)

import datetime
import math
import time
import json
from youtube_transcript_api import YouTubeTranscriptApi
import tiktoken
import faiss
import numpy as np
import asyncio
import ipywidgets
from IPython.display import display, HTML
enc=tiktoken.get_encoding("cl100k_base")


keys=json.load(open("working_folder/keys.json"))

# Setup anthropic
import anthropic
client = anthropic.Anthropic(api_key=keys["anthropic"])

# Setup openai
from openai import OpenAI
os.environ["OPENAI_API_KEY"]=keys["openai"]
openai_client = OpenAI(api_key=keys["openai"])

# Get transcript, Define video

In [32]:
# Get raw transcript
video_id="FC3giY7YDAQ"
#video_id="hkOI46Ai4g8"
video_id="YZuSc-EnI3o"
if os.path.isdir("working_folder")==False:
    os.mkdir("working_folder")
if os.path.isdir("working_folder/"+video_id)==False:
    os.mkdir("working_folder/"+video_id)

raw_transcript=YouTubeTranscriptApi.get_transcript(video_id)
transcript=""
nearest_times={}
for m in raw_transcript:
    print(m['text'], m["start"])
    transcript+=str(m['text'])+"\n"
    nearest_times[len(transcript)]=m["start"]
    # if len(transcript)>50000:
    #     break

# save as transcript.txt
with open("working_folder/transcript.txt","w") as f:
    f.write(transcript)

e 28.96
e 58.92
e 88.92
e 118.84
e 148.84
e 178.84
e 208.76
gotta get some L's in chat for Dune 2 215.2
can we get some L's dun to in chat 217.439
please oh my 220.64
God 238.72
Jesus Christ what the what the [ __ ] what 241.239
am I looking at what the [ __ ] did I 244.28
click holy 248.84
[ __ ] is that man biting that dude's 252.519
penis 254.879
off why what the [ __ ] what for what 267.28
reason 298.639
for 328.639
W the L thanks for 20 331.12
bucks dun 2 is way better than the first 335.72
can you admit 338.24
that uh dunu was way better than the 339.52
first one yes that is 342.28
true indeed 347.759
indubitably 358.639
fresh math Rock cat jam party 386.919
[Music] 391.09
[Applause] 398.0
[Music] 399.11
[Applause] 403.76
[Music] 405.03
oh good it's cute 418.56
trying to think if I should go now if we 422.599
should wait till after our tree talk I 423.919
figured out why I figured out why I 425.36
don't like these movies and you know 427.36
what I think it all ties into everythin

# Intialize Variables and Functions

In [3]:
# Cost calculation
input_cost=0
output_cost=0
total_cost=input_cost+output_cost
def get_cost(input_text,output_text):
    global input_cost
    global output_cost
    global total_cost
    input_cost+=len(enc.encode(input_text))*(3/1000000.0)
    output_cost+=len(enc.encode(output_text))*(15/1000000.0)
    total_cost=input_cost+output_cost

# make vector db
def make_vector_db(text_document):
    # Chunk text
    chunk_size=1000
    text_chunks=[text_document[i:i+chunk_size] for i in range(0, len(text_document), chunk_size)]
    print("Number of chunks: ",len(text_chunks))

    # Generate embeddings
    model="text-embedding-3-large"
    embeddings = []
    for chunk in text_chunks:
        response = client.embeddings.create(input=chunk, model=model)
        embeddings.append(response.data[0].embedding)
    embeddings=np.array(embeddings)
    print("Finished generating embeddings")

    # Make vector db
    vector_db=faiss.IndexFlatL2(embeddings.shape[1])
    vector_db.add(np.array(embeddings))

    # return text chunks and vector db
    return(text_chunks, embeddings, vector_db)

# Async function to fetch embeddings
async def fetch_embeddings_async(text_chunks, model):
    model="text-embedding-3-large"
    async def fetch_embedding(chunk):
        # Simulate an async call to the embeddings API
        return await asyncio.to_thread(openai_client.embeddings.create, input=chunk, model=model)

    responses = await asyncio.gather(*(fetch_embedding(chunk) for chunk in text_chunks))
    embeddings = [response.data[0].embedding for response in responses]
    return np.array(embeddings)

# make vector db
async def make_vector_db_fast(text_document):
    # Chunk text
    chunk_size=1000
    text_chunks=[text_document[i:i+chunk_size] for i in range(0, len(text_document), chunk_size)]
    print("Number of chunks: ",len(text_chunks))

    # Generate embeddings
    model="text-embedding-3-large"
    embeddings=await fetch_embeddings_async(text_chunks, model)
    print("Finished generating embeddings")

    # Make vector db
    vector_db=faiss.IndexFlatL2(embeddings.shape[1])
    vector_db.add(np.array(embeddings))

    # return text chunks and vector db
    return(text_chunks, embeddings, vector_db)


# vector search
def search_vector_db(query, k, vector_db):
    # Generate query embedding
    query_embedding = openai_client.embeddings.create(input=query,model="text-embedding-3-large").data[0].embedding
    query_embedding_np = np.array(query_embedding).astype('float32').reshape(1, -1)

    D, I = vector_db.search(query_embedding_np, k)
    return (D,I)

# Get time of transcript
def get_time_at_length_transcript(length):
    i=0
    while nearest_times.get(length-i,None)==None:
        i+=1
        if (length-i)<=0:
            return list(nearest_times.values())[0]

    return(nearest_times[length-i])

# Put hyperlink to time in text
def convert_to_html(text,start_second, index):
    lines = text.split('\n')
    len_so_far=0
    html_lines = []
    for line in lines:
        line = line.strip()
        len_so_far+=len(line)
        if line:
            time_at_hyperlink=get_time_at_length_transcript(index*1000+len_so_far)-3
            if time_at_hyperlink<0:
                time_at_hyperlink=0
            line+=str(index*1000+len_so_far)
            hyperlinked_line = f'<a href="https://www.youtube.com/watch?v={video_id}#t={time_at_hyperlink}s"c">{line}</a>'
            html_lines.append(hyperlinked_line)

        

    html_text = '<br>'.join(html_lines)
    return html_text

# Produce embedding search index

In [38]:
# Produce text chunks and vector db
text_chunks, embeddings, vector_db= await make_vector_db_fast(transcript)#[:10000])

# Save json text chunks
text_chunks_dict={}
index=0
for t_chunk in text_chunks:
    text_chunks_dict[str(index)]={"text":t_chunk, "start":get_time_at_length_transcript(transcript.find(t_chunk)), "end":get_time_at_length_transcript(transcript.find(t_chunk)+len(t_chunk))}
    index+=1
json.dump(text_chunks_dict,open("working_folder/"+video_id+"/text_chunks_dict.json","w"))

# Save vector db
faiss.write_index(vector_db, "working_folder/"+video_id+"/vector_db.index")

Number of chunks:  386
Finished generating embeddings


## Search index

In [41]:
#load from text_chunks_dict
text_chunks_dict=json.load(open("working_folder/"+video_id+"/text_chunks_dict.json"))
D, I=search_vector_db("Remove moles",5,vector_db)

thtml=convert_to_html(text_chunks_dict[str(I[0][0])]["text"],text_chunks_dict[str(I[0][0])]["start"],I[0][0])
display(HTML(thtml))

# Incremental Summarization


In [4]:
increment_chars=10000
char_start_index=0
segments=3
model_responses=[]

model_prompt="Your purpose is to take a transcript from a youtube streamer named Destiny and give a synopsis of the content and the sentiment/takes of the speaker. Include all of the topics even if they are covered briefly instead of just covering the main topic."

get_response=False


# get a certain number of segments
while (len(model_responses)<segments) and ((char_start_index+increment_chars)<=len(transcript)):
    input_transcript=transcript[char_start_index:char_start_index+increment_chars]

    conv_messages=[{"role": "user", "content": "Transcript: "+input_transcript}]
    bot_response=""

    # display start and endtime
    start_second_raw=get_time_at_length_transcript(char_start_index)
    hours = math.floor(start_second_raw / 3600)
    minutes = math.floor((start_second_raw % 3600) / 60)
    seconds = start_second_raw % 60

    # calculate end time
    end_second_raw=get_time_at_length_transcript(char_start_index+increment_chars)
    hours_end = math.floor(end_second_raw / 3600)
    minutes_end = math.floor((end_second_raw % 3600) / 60)
    seconds_end = end_second_raw % 60

    print(f"Start time {int(hours):02d}:{int(minutes):02d}:{seconds:06.3f}  End time {int(hours_end):02d}:{int(minutes_end):02d}:{seconds_end:06.3f}")

    

    if get_response:
        with client.messages.stream(
            max_tokens=2024,
            system=model_prompt,
            messages=conv_messages,
            #model="claude-3-opus-20240229",
            model="claude-3-sonnet-20240229",
        ) as stream:
            for text in stream.text_stream:
                bot_response+=text
                print(text, end="", flush=True)
        print()

        get_cost(input_transcript+model_prompt,bot_response)
        print("Cost: ",total_cost)

        model_responses.append({"bot": bot_response,"transcript": input_transcript})
    
    char_start_index+=increment_chars-300
        


Start time 00:00:28.960  End time 00:17:09.480
Start time 00:16:46.000  End time 00:29:59.640
Start time 00:29:40.640  End time 00:40:36.800
Start time 00:40:16.960  End time 00:50:49.079
Start time 00:50:37.200  End time 00:59:29.680
Start time 00:59:14.960  End time 01:09:23.080
Start time 01:08:56.440  End time 01:19:41.159
Start time 01:19:30.239  End time 01:28:14.960
Start time 01:27:59.440  End time 01:39:10.360
Start time 01:38:47.760  End time 01:49:00.560
Start time 01:48:34.960  End time 02:04:14.599
Start time 02:03:54.440  End time 02:14:01.280
Start time 02:13:37.719  End time 02:21:30.240
Start time 02:21:15.920  End time 02:29:10.760
Start time 02:28:55.359  End time 02:37:23.200
Start time 02:37:10.439  End time 02:47:11.240
Start time 02:47:00.680  End time 02:54:15.600
Start time 02:54:03.600  End time 03:01:51.359
Start time 03:01:42.960  End time 03:08:27.120
Start time 03:08:14.840  End time 03:14:47.920
Start time 03:14:37.760  End time 03:22:46.080
Start time 03

# Async Summarization

In [11]:
# Produce summaries for each transcript segment asynchroneously
async def get_claude_responses(input_data):
    input_transcripts=[]
    input_indexes=[]
    for input_ in input_data:
        input_transcripts.append(input_[0])
        input_indexes.append(input_[1])
    

    model_prompt="Your purpose is to take a transcript from a youtube streamer named Destiny and give a synopsis of the content and the sentiment/takes of the speaker. Include all of the topics even if they are covered briefly instead of just covering the main topic."

    # Def synchronous api call
    def fetch_response(transcript,index):
        conv_messages=[{"role": "user", "content": "Transcript: "+transcript}]
        bot_response=""
        while True:
            bot_response=""
            try:
                with client.messages.stream(
                        max_tokens=2024,
                        system=model_prompt,
                        messages=conv_messages,
                        model="claude-3-sonnet-20240229",
                    ) as stream:
                        for text in stream.text_stream:
                            bot_response+=text
                break
            except:
                print("Error in api call waiting 10 seconds",index)
                time.sleep(10)
                print("Retrying api call ",index)

        return [bot_response,index]
    
    # Create thread to run api call
    async def thread_fetch(transcript,index):
        thread = await asyncio.to_thread(fetch_response, transcript,index)
        return(thread)

    # Gather all the responses
    responses = await asyncio.gather(*(thread_fetch(in_data[0],in_data[1]) for in_data in input_data))
    return(responses)


In [12]:
# Setup Variables
increment_chars=10000
char_start_index=0
#segments=3
segments=100
model_responses=[]
tasks=[]
index=0

# get a certain number of segments
while (len(model_responses)<segments) and ((char_start_index+increment_chars)<=len(transcript)):
    input_transcript=transcript[char_start_index:char_start_index+increment_chars]
    conv_messages=[{"role": "user", "content": "Transcript: "+input_transcript}]
    bot_response=""

    # display start and endtime
    start_second_raw=get_time_at_length_transcript(char_start_index)
    hours = math.floor(start_second_raw / 3600)
    minutes = math.floor((start_second_raw % 3600) / 60)
    seconds = start_second_raw % 60

    # calculate end time
    end_second_raw=get_time_at_length_transcript(char_start_index+increment_chars)
    hours_end = math.floor(end_second_raw / 3600)
    minutes_end = math.floor((end_second_raw % 3600) / 60)
    seconds_end = end_second_raw % 60

    sf_str=f"Start time {int(hours):02d}:{int(minutes):02d}:{seconds:06.3f}  End time {int(hours_end):02d}:{int(minutes_end):02d}:{seconds_end:06.3f}"
    
    model_responses.append({"bot": "","transcript": input_transcript,"time_string":sf_str,"char_start_finsih_indexes":[char_start_index,char_start_index+increment_chars], "index":index})

    index+=1
    char_start_index+=increment_chars-300



# get approximate cost of run
prev_cost=total_cost
for m in model_responses:
    get_cost(m["transcript"],"a b c"*200)
print("Approximate cost: ",total_cost-prev_cost, "  Number of segments: ",len(model_responses))

# Get user decision to proceed
proceed=input("Proceed with run? (y/n): ")
if proceed.lower()!="y":
    print("Run cancelled")
else:
    bot_responses=await get_claude_responses([[m["transcript"],m["index"]] for m in model_responses])
    for i in range(len(bot_responses)):
        model_responses[i]["bot"]=bot_responses[i][0]
        get_cost(model_responses[i]["transcript"],model_responses[i]["bot"])
    print(total_cost)

    # save model responses to json
    json.dump(model_responses,open("working_folder/"+video_id+"/model_responses.json","w"))


Approximate cost:  0.5281859999999994   Number of segments:  39
Error in api call waiting 10 seconds 26
Error in api call waiting 10 seconds 24
Error in api call waiting 10 seconds 27
Error in api call waiting 10 seconds 32
Error in api call waiting 10 seconds 31
Error in api call waiting 10 seconds 29
Retrying api call  26
Retrying api call  24
Retrying api call  27
Retrying api call  32
Retrying api call  31
Retrying api call  29
2.0528039999999983


# Cost description
0.003 per kt input 
0.015 per kt output

typical input is a round 2.5kt, typical output is around 0.35kt
input $0.0075 and output $0.005 for 1 summary
input $0.11 and output $0.075 for 14 summaries totalling 3 hrs with about 15 minutes per segment
total $0.18 for 3 hrs

In [14]:
mrsl=json.load(open("working_folder/"+video_id+"/model_responses.json"))

In [45]:
# get user query
query=input("What you want")

# load from text_chunks_dict
text_chunks_dict=json.load(open("working_folder/"+video_id+"/text_chunks_dict.json"))
# load vector db
vector_db=faiss.read_index("working_folder/"+video_id+"/vector_db.index")
# search vector db
D, I=search_vector_db(query,5,vector_db)

# display result
html0=convert_to_html(text_chunks_dict[str(I[0][0])]["text"],text_chunks_dict[str(I[0][0])]["start"],I[0][0])
html1=convert_to_html(text_chunks_dict[str(I[0][1])]["text"],text_chunks_dict[str(I[0][1])]["start"],I[0][1])
html_all=html0+"<br><br>"+html1
display(HTML(html_all))

In [16]:
all_summaries=""
for mr in mrsl:
    print(mr["time_string"])
    print(mr["bot"],"\n")
    all_summaries+=mr["time_string"]+"\n"+mr["bot"]+"\n\n"


Start time 00:00:28.960  End time 00:17:09.480
Based on the transcript, Destiny was discussing his thoughts and criticisms of the Dune movies, particularly Dune: Part Two. Here's a synopsis of the key points he made:

Topics:
- Expressing dislike for Dune: Part Two, calling it boring
- Critiquing the lack of compelling characters he cares about 
- Comparing it unfavorably to Game of Thrones, which he felt had much more interesting and well-developed characters
- Enjoying the world-building and lore more in Part Two compared to Part One
- Finding some of the romantic dialogue and delivery by actors cringeworthy  
- Appreciating certain actors like Oscar Isaac as the father and the brutish bodyguard character
- Liking Paul's story arc of wrestling with his identity/destiny, but disliking the Paul character himself
- Thinking the worldbuilding was good but not enough to compensate for the lack of engaging characters

Sentiment/Takes:
- Destiny seems to greatly value compelling, fleshed-ou

# Meta summarization

In [51]:
# print expected cost and ask if user wants to proceed
print("Expected cost: ",len(enc.encode(all_summaries))*(3/1000000.0))
proceed=input("Proceed with run? (y/n): ")
if proceed.lower()!="y":
    print("Run cancelled")
else:
    meta_model_prompt="Your purpose is to take a conglomerate of summaries and compile it into one conglomerate which provides a comprehensive and effective way of knowing what things were talked about in the collection of summaries. The summaries are off of a youtube video transcript of a youtube streamer named Destiny. You should do two parts, main or big topics that were talked about as a main focus or for a long period and another section of smaller details or topic that were covered briefly. These should be two large sections, each may be 300-500 words. In total you need to write around 1000 words. Be sure to include a lot of detail and be comprehensive to get to that 1000 word mark."

    bias_injection="Some information about me the user, I like technology, specifically software but technology generally, I am interested in full democracy, I am probably a bit right leaning and am curious about critiques to conservative views, I am curious about science, and I enjoy humor. "

    if bias_injection!="":
        meta_model_prompt+=" If the user states information about them, cater the summary to their interests."

    # Meta summary
    bot_response=""
    with client.messages.stream(
            max_tokens=4096,
            system=meta_model_prompt,
            messages=[{"role":"user", "content": bias_injection+"Collection of summaries for the video/transcript: "+all_summaries}],
            model="claude-3-sonnet-20240229",
        ) as stream:
            for text in stream.text_stream:
                bot_response+=text
                print(text, end="", flush=True)
    # save bot response as text file
    with open("working_folder/"+video_id+"/meta_summary.txt","w") as f:
        f.write(bot_response)


Expected cost:  0.037515


Main or Big Topics Covered:

1. Critiques of the Dune movies, especially Part Two:
Destiny expressed significant disappointment with the Dune movies, particularly Part Two. He found the characters to be stiff, lacking in personality and emotional range, making it difficult for him to become invested in their arcs. Despite praising the visual effects, worldbuilding, and certain elements like Paul's struggle with destiny, the overall lack of compelling characters was a major flaw in his view. He drew contrasts to other acclaimed works like Game of Thrones, where he felt the characters were far more fleshed out and memorable.

2. Perspectives on Sexual Misconduct Allegations:  
The transcript covered Destiny's responses to various allegations of sexual misconduct or impropriety made against him and others in the online/streaming community. In several instances, he pushed back against the framing or details provided, arguing that some encounters depicted as non-consensual were in fact cons

In [40]:
print("Before:",len(enc.encode(transcript)),"  After:",len(enc.encode(bot_response)))
print(bot_response)

NameError: name 'bot_response' is not defined

# Fetch chat messages 

In [33]:
from chat_downloader import ChatDownloader

url = 'https://www.youtube.com/watch?v='+video_id
chat = ChatDownloader().get_chat(url)       # create a generator
c_count=0
all_chat_messages=[]
all_chat_messages_str=""
for message in chat:                        # iterate over messages
    #chat.print_formatted(message)   
    chat_message_formatted="User: "+message["author"]["name"] +", "+message["time_text"]+", Message: "+message["message"]
    all_chat_messages.append(chat_message_formatted)
    all_chat_messages_str+=chat_message_formatted+"\n"
    if c_count%100==0:
        print(chat_message_formatted) 
    c_count+=1
print(c_count)

# save chat string and messages
with open("working_folder/"+video_id+"/chat.txt","w", encoding='utf-8') as f:
    f.write(all_chat_messages_str)
json.dump(all_chat_messages,open("working_folder/"+video_id+"/chat.json","w"))
print("saved chat")


User: Fung Dark, 0:23, Message: Yooooo
User: FLAGELLANT, 2:14, Message: troll troll troll title!
User: Viktor V, 3:54, Message: L
User: Wow, 4:33, Message: :_destinyFeelsGood:
User: The Sleepy Gamer, 6:01, Message: This is what happens when you don’t have solid epistemic grounding 
User: 921, 7:56, Message: if they ever make god emperor of dune i want destiny to play leto the second while wearing a gigantic fatsuit
User: bobono, 9:49, Message: did you watch it in IMAX
User: lostukulele, 11:35, Message: They are on screen long enough to create that
User: Chelts, 13:13, Message: it's harder to character develop in movies vs series
User: Gloria Walmert, 14:18, Message: it's not a romance it's her watching her lover change into a power hungry emperor
User: Piccolo, 16:06, Message: dune 1 6.5/10 , 4 hours of build up, dune 2 7.5/10
User: Spiero, 17:42, Message: I liked that every player was scheming.
User: Nihm420, 19:42, Message: ahhhhh come on now you loved it
User: Alex Kiecker, 21:14, M

UnicodeEncodeError: 'charmap' codec can't encode characters in position 9474-9475: character maps to <undefined>

In [54]:
cuml={}
for cm in all_chat_messages:
    usr=str(cm.split(",")[0].split("User: ")[1])
    cuml[usr]=cuml.get(usr,0)+1

# go through key and val of cuml
rcuml={}
for key, val in cuml.items():
    rcuml[val]=rcuml.get(val,[])+[key]

In [58]:
rcuml[54]

['CASEY Andersen', 'HumanGarbage', 'E Martinez']

In [52]:
cmcl=list(cuml.values())
cmcl.sort(reverse=True)

In [59]:
print(len(cmcl))

3145


In [62]:
# get sum of first 20% of cmcl and total sum
sum_20=0
sum_total=0
for i in range(len(cmcl)):
    sum_total+=cmcl[i]
    if i<(len(cmcl)*0.2):
        sum_20+=cmcl[i]
print(sum_20)
print(sum_total)

15460
20698


In [63]:
15460/20698

0.7469320707314716

In [53]:
print(cmcl)

[212, 185, 155, 154, 153, 150, 138, 135, 126, 123, 118, 112, 109, 109, 106, 103, 101, 100, 98, 97, 94, 92, 91, 90, 90, 89, 89, 82, 82, 81, 80, 79, 78, 74, 72, 71, 70, 68, 68, 67, 65, 64, 63, 62, 61, 61, 61, 60, 60, 59, 59, 58, 58, 57, 56, 56, 55, 55, 54, 54, 54, 53, 51, 51, 50, 49, 48, 48, 46, 46, 45, 45, 45, 45, 44, 44, 44, 43, 43, 43, 43, 43, 42, 42, 42, 42, 42, 42, 41, 41, 41, 41, 40, 40, 40, 40, 39, 39, 39, 39, 39, 38, 38, 37, 37, 37, 37, 36, 36, 36, 36, 36, 36, 36, 36, 35, 35, 35, 35, 34, 34, 34, 34, 34, 34, 33, 33, 33, 33, 33, 33, 33, 32, 32, 32, 31, 31, 31, 31, 31, 31, 31, 30, 30, 30, 30, 30, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 28, 28, 28, 28, 28, 27, 27, 27, 27, 27, 27, 27, 27, 27, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 24, 24, 24, 24, 24, 24, 24, 24, 24, 23, 23, 23, 23, 23, 23, 23, 23, 22, 22, 22, 22, 22, 22, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 2

## Process chat

In [4]:
# Chat summary
bot_response=""
chat_summary_prompt="You are meant to summarize serious messages in chat and get a general sentiment of the chat as well as pull out the repeated serious sentiments or serious questions or on topic messages. The user will give you all the chat messages with username\nMessage."
with client.messages.stream(
        max_tokens=4096,
        system=chat_summary_prompt,
        messages=[{"role":"user", "content": "Here is the chat: "+chat_messages}],
        model="claude-3-sonnet-20240229",
    ) as stream:
        for text in stream.text_stream:
            bot_response+=text
            print(text, end="", flush=True)

Here are the key points I gathered from summarizing the serious messages in this chat:

- There is discussion around whether Biden should debate Trump, with some arguing he should to show he's up for the challenge, and others saying he shouldn't give Trump a platform since he's seen as a criminal by some.

- A few people are asking why Biden may refuse to debate Trump, speculating it's because Biden would perform poorly in a debate.

- Some are calling out the banning/moderation of certain users as being too harsh or authoritarian.

- There is some curiosity around Destiny's debate/conversation style and how engaged he is with the active chatters.

- A couple serious questions were asked, like whether Biden refusing debates sets a concerning precedent, and asking Destiny's opinion on Biden not debating Trump.

The overall sentiment seems a bit divided on the debate topic, with arguments on both sides. There is also some pushback against the moderation happening in the chat. A few on-to

In [1]:
chat_messages="""
Bolo Leo
​​they all watching sw on some hidden screen

Delightful Electric Car
New member
​​Behahahaha. 🫠

will 2022
​​since buck was actually a democrat in disguise

Tiny Metal Knight
​​Narrator: They didn't learn

STRIKER
​​based

E Martinez
Member (1 year)
​​you will learn to love me

Dr. Smell Zone
Member (1 year)
​​this seems fake

Spiero
Member (6 months)
​​CONCLOODE

The Internet Summarized
​​the bans will continue until moral improves

MirGravity
Member (1 month)
​​AYAYAS

Sage Emerald
​​LETS GO

Se7en Sinner
​​actually based

Leo Charles
Member (1 year)
​​destinyPepeLaugh

Bajango
​​big ben na

The Jonavic
Member (6 months)
​​spammers should be banned for a year

Norm’s Ghost
Member (6 months)
​​destinyGIGACHADI am incapable of learning via consequences

Nathan
​​good luck if u tune in late

hexigog
​​@Destiny I heard that Biden is refusing to debate trump. is that true? and if so do you think it's a good idea/what would be the justification?

Kevinloks
​​GET THAT ASS BAN

Egusi Soup
Member (1 year)
​​I mean it workeddestinyGIGACHAD

Kriswin
​​the purge

xghostyyx
​​I AM A MARTYR

Delightful Electric Car
New member
​​Someone use the MEME!!!

Alex Neumann
Member (1 month)
​​it's joever

Moonface
​​Out here banning your content generators lulz

Bean E
Member (2 months)
​​You forgot to say, I will teach you

Nolan Tracy
​​perks of yt chat no responsibilities

Delightful Electric Car
New member
​​Dr K! Looking sexy.

E Martinez
Member (1 year)
​​face-orange-raised-eyebrow

Joe dapro
​​Tyrannical Destiny are BASED

Guacamolius
Member (1 month)
​​cause you give them shout-outs, acknowledge their links or messages and make a big deal about it. that's why. but ok

Nolan
Member (2 months)
​​HAHAHAH

lOcIEe
Member (2 months)
​​face-orange-biting-nails

Darkpaolos
​​genociding chatters

Kristjan Birnbaum
​​dog-walker moment

Rick Montana
Member (1 year)
​​Gl

Alex Neumann
Member (1 month)
​​destinyPepeLaugh

IWSS
​​Baaased

envi
Member (6 months)
​​Harvard Psychiatrist? Mr Girl will be mad

will 2022
​​only thing i block is spam

Se7en Sinner
​​high risk, high reward

Delightful Electric Car
New member
​​Who told Dr K to be looking like that. 🥵

will 2022
​​its annoying

Big sundays
​​1984

deref
​​1984

Josh Flynn
Member (1 year)
​​face-blue-wide-eyes

krinkerlsjhkg
​​1984

Nolan Tracy
​​@Nolan Change your name

gr666eg
​​I like big bans and I cannot lie

Johan Nilsson
​​I was thinking about some tool for you streamer check somthing has been noted.

Jim Searsino
​​thank god he doesnt read youtube :)

Dennis
​​laying flat, just like saddam hussein in his hidey-hole

Nolan Tracy
​​Wtf

Zion Dignan
​​Do I have to spam on disc or YouTube to get him to see a link?

krinkerlsjhkg
​​literally

Keelr
​​True

Audaylon
​​trump won every won too

Norm’s Ghost
Member (6 months)
​​HE BETTER NOT DEPRIVE US OF THAT

Benny
​​@Destiny Do you feel more loose when you're on the Ice coffee Pod ?

FastAndBulbous
​​based

KaylaMoree
​​He should refuse cause why would he debate a criminal?

Uncanny Beef
Member (1 year)
​​they already have debates scheduled in Sept right?

Rick Montana
Member (1 year)
​​Debate rapist (in civil court)

Gabreyes093
​​Yeah. Just keep trolling him

What's the Argument?
​​Anti work; oh you mean just doing exactly what your were contracted to be paid for?

E Martinez
Member (1 year)
​​nikki haley😍😍

Bushcraft all day
Member (2 months)
​​he doesnt want to see biden get wrecked

Spiero
Member (6 months)
​​Ban him destinyOMEGALUL

Anime Fan
​​Have mercy

Josh N.
Member (2 months)
​​oh, we know

KaylaMoree
​​I pray he doesn’t get bullied into the debate cause fuck that

deref
​​TRUE

Josh Flynn
Member (1 year)
​​destinyPepe

Cheddar Man
New member
​​Biden will look like a fool if he debates ANYONE. That is the only reason why we are even talking about it

Unseen
​​Thinning the stupid herd, based

Guacamolius
Member (1 month)
​​we can guess destinyCoomer

Kriswin
​​"pay attention to me"

Egusi Soup
Member (1 year)
​​that's the point. No one has any idea

arcadebishop
Member (6 months)
​​destinyOMEGALULdestinyOMEGALUL

Ares Martinez
New member
​​Fallout new Vegas??

TheSeriouslyCasual
​​FINISH HIM

envi
Member (6 months)
​​!mutelinks on

Oliver
​​lol

krinkerlsjhkg
​​REEEEE

keys
Member (1 month)
​​how does destiny keep up with so many chatters? Is he on meth?

arcadebishop
Member (6 months)
​​yooooo

Eugene A
Member (2 months)
​​i wish

Mike Suarez
​​Such cope lol. Trump didn't debate anyone cause he knew it was beneath him. Biden should debate his 1 adversary

Bajango
​​no debates seems like a weird precedent to set

Bolo Leo
​​yes we do

Conservative Gaming
Member (2 months)
​​destinyGodstiny

King of Kings
​​Jesus !

Seb
Member (2 months)
​​We do its called Youtube chat

The Jonavic
Member (6 months)
​​right to his brain

Josh Flynn
Member (1 year)
​​destinyCoomer

Szzrhn
​​oneguy

Kurosai
​​calm down babe

OMW
Member (2 years)
​​destinyCoomer

Epsilon The Advent
​​@Kurosai woah wtf are you doing here, love your vids lol

Hal Courteney
​​?? why is that spooky

Josh N.
Member (2 months)
​​spooky scary

LittleTallGuy
​​show of good faith, or a trojan horse?

Tyler Russell
​​That’s just their brand

Lucy Archangel
​​army expeditionary force

Seb
Member (2 months)
​​maybe its part of Biden's master plan

Gonzo
​​Pls talk to the bonger"""